### Libraries and INitializing PySpark

In [1]:
from pyspark import SparkContext

# sc is short for spark context
sc = SparkContext(appName="IntroToSpark")

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
taxi_entry = sc.parallelize(['2009-01-01', 0, 0, 24])

In [ ]:
print(taxi_entry)

In [ ]:
ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [ ]:
from pyspark import SparkContext

sc = SparkContext(appName="IntroTo Spark")# < write code here >
weather_entry = sc.parallelize(['2009-01-01', 15.1, 26.1])# < write code here >
print(weather_entry.take(3))

In [ ]:
from pyspark.sql import SparkSession

APP_NAME = 'sampleApp'

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

### DataFrames in PySpark

In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

taxi_df = pd.read_csv('/datasets/pickups_terminal_5.csv')
taxi = spark.createDataFrame(taxi_df)

print(taxi)

In [ ]:
[Row(date='2009-01-01', hour=0, minute=0, pickups=24.0), 
Row(date='2009-01-01', hour=0, minute=30, pickups=35.0), 
Row(date='2009-01-01', hour=1, minute=0, pickups=25.0), 
Row(date='2009-01-01', hour=1, minute=30, pickups=25.0), 
Row(date='2009-01-01', hour=2, minute=0, pickups=16.0)]

In [ ]:
# you can break the line using the symbol '\'
spark = SparkSession.builder.appName(APP_NAME) \
        .config('spark.ui.showConsoleProgress', 'false') \
        .getOrCreate()

In [ ]:
# format='csv' - specify file format
# header='true' - specify that there is a header (column names) in the file
# inferSchema='true' - specify that data types should be inferred
taxi = spark.read.load('/datasets/pickups_terminal_5.csv', 
                       format='csv', header='true', inferSchema='true')

In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

taxi = spark.read.load('/datasets/pickups_terminal_5.csv', format='csv', header='true', inferSchema='true')# < write code here >

print(taxi.show(5))# < write code here >

In [ ]:
print(taxi.count()) #total number of rows in the dataframe

In [ ]:
print(taxi[['date', 'hour', 'minute']].show(5)) #susbset columns